In [21]:
import torch
import torch.nn as nn
from MyDataset import TextDataset
from my_tokenizer import tokenizer
from model import Seq2SeqModel
import pandas as pd

data = pd.read_pickle("mydata.pkl")
max_length = 256

In [67]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dataset = TextDataset(data, tokenizer, max_length)

special_tokens = {"CLS": 3, "eos": 2, "pad": 1, "unk": 0}
dataset.tokenizer.stoi.update(special_tokens)
n_words = dataset.get_vocab_len()

model = Seq2SeqModel(vocab_size=n_words, embed_size=128, hidden_size=256, num_layers=1)
model.load_state_dict(torch.load('GPT_model.pth', weights_only=True))

from torch.utils.data import DataLoader
data_loader = DataLoader(dataset, batch_size=1, shuffle=True)


def apply_temperature(logits, temperature=0.5):
    logits = logits / temperature
    return torch.nn.functional.softmax(logits, dim=-1)

def top_k_sampling(logits, k=10):
    # Take the logits from the model output and apply top-k sampling
    top_k_values, top_k_indices = torch.topk(logits, k)
    top_k_probs = torch.nn.functional.softmax(top_k_values, dim=-1)
    sampled_token = torch.multinomial(top_k_probs, 1)
    return top_k_indices[0, sampled_token]

def generate_text(model, title, tokenizer, max_len=300):
    model.eval()
    
    # Prepare title input
    title_tokens = tokenizer(title)
    title_tokens = torch.tensor([dataset.tokenizer.stoi[token] for token in title_tokens]).long()
    title_tokens = torch.nn.functional.pad(title_tokens, (0, 256 - title_tokens.shape[0] - 1), value=special_tokens["pad"])
    title_tokens = torch.nn.functional.pad(title_tokens, (0, 1), value=special_tokens["eos"])


    generated_text = []
    input_text = torch.tensor([[special_tokens["CLS"]]]).long().to(device)  # Starting input token

    title_tokens = title_tokens.unsqueeze(0).to(device)
    for _ in range(max_len):
        output = model(title_tokens, input_text)
        #next_token = torch.argmax(output[:, -1, :])
        #generated_text.append(next_token.item())
        
        # Get logits for the last token in the sequence
        logits = output[:, -1, :]

        logits = apply_temperature(logits)
        sampled_token = top_k_sampling(logits)
        generated_text.append(sampled_token.item())


        # Stop if <eos> token is generated
        if sampled_token == special_tokens["eos"]:
            break

        # Prepare next input
        input_text = torch.cat([input_text, torch.tensor([[sampled_token.item()]])], dim=1)


    decoded_text = ""
    keys = list(dataset.tokenizer.stoi.keys())
    for idx in generated_text:
        if idx <= len(dataset.tokenizer.stoi):
            decoded_text += " " + keys[idx]
        else:
            decoded_text += " <unk>"

    return decoded_text.strip()

# Example usage
title = "Věřím?"
generated_text = generate_text(model, title, tokenizer)
print("TEXT:", generated_text)

TEXT: jedinci sebemenšího 1 kdyby absolvuje silným vagón živého vagón docházka články jedinci problematiku problematiku vagón absolvuje sebemenšího milionářem nabývá 11 jakou 1 ženami 1 nebojíme jedinci verš vagón absolutní jakou 1 ženami junák silným dobra dobra živého 11 11 toho 1 kipling 11 jedinci toho absolvuje přestáváme 1 jakou absolutní jakou jedinci toho 1 ženami jedinci absolutní jedinci 1 ženami vagón silným jedinci absolutní nabývá taková vagón živého 1 ženami 1 ženami jedinci verš taková absolutní jakou 11 absolvuje byl 1 jedinci 11 alfreda absolvuje protějšky toho absolutní absolvuje jím 11 jím sebemenšího nade 11 absolvuje přestáváme 1 nebojíme ideál 11 vagón jedinci toho silným taková 1 jedinci vagón 1 ženami vagón jedinci vagón dnešním absolutní jakou jedinci 11 ideál toho 11 jakou jedinci 1 nebojíme jedinci 11 absolvuje byl 1 alfreda toho absolvuje protějšky absolutní jedinci absolutní absolvuje protějšky 11 absolvuje toho neobvyklého 11 jakou 11 - silným toho toho má